In [1]:
import os
import sys
import subprocess
import conda
import importlib
import socket
import getpass
import datetime
from functools import reduce

run = lambda *cmd: subprocess.run(cmd, capture_output=True)

### Helper Functions

In [2]:
def check_version(name, failed=False):
    version = run(name,'--version')
    if version.stderr:
        install = run('conda','install','-y',name)
        return install
    else:
        return version

def check_import(name, failed=False):
    try:
        m = importlib.import_module(name)
        res = m.__version__
    except ImportError as e:
        if not failed:
            run('pip','install',name)
            return check_import(name, failed=True)
        else:
            res = repr(e)
    return res

## Check conda path

The installation path of anaconda cannot contain any space characters (```' '```).

In [3]:
conda_path_ok = conda.CONDA_PACKAGE_ROOT.find(' ') < 0

In [4]:
conda_version = run('conda','--version')

In [5]:
# check version
if int(''.join(conda_version.stdout.decode('utf-8').strip()[-5:].split('.'))) < 400:
    conda_update = run('conda','update','-y','conda')
else:
    conda_update = None

In [6]:
conda = run('conda','info')

In [7]:
# conda_update_all = run('conda','update','-y','--all')

## Check conda installs

In [8]:
python = check_version('python')
pip = check_version('pip')
git = check_version('git')
jupyter = check_version('jupyter')

In [9]:
%%capture vscode
# VS Code triggers FileNotFoundError when "run('code')"  on Windows
!code --version

## Check python packages

In [10]:
matplotlib = check_import('matplotlib')
numpy = check_import('numpy')
pandas = check_import('pandas')
pip_py = check_import('pip')
pytest = check_import('pytest')
hypothesis = check_import('hypothesis')

In [11]:
%%capture matplotlib_backend
%matplotlib

## Print Report

In [12]:
def write_report(*contents):
    with open("syscheck-report-{}-{}-{}.txt".format(
                socket.gethostname(),
                getpass.getuser(),
                datetime.datetime.now().date().isoformat()
                )
              , 'w') as f:
        
        for line in contents:
            if line:
                f.write(line + '\n')
        else:
            f.write('\n'+hex(hash(reduce(lambda a,b: a+b, contents))))

In [13]:
write_report(
    'Report',
    '------',
    '',
    '- Conda path clean: {}'.format(conda_path_ok),
    "- Conda cli:",
    conda.stdout.decode('utf-8'),
    conda.stderr.decode('utf-8'),
#     conda_update_all.stdout.decode('utf-8'),
#     conda_update_all.stderr.decode('utf-8'),
    "\n- Python cli:",
    python.stdout.decode('utf-8'),
    python.stderr.decode('utf-8'),
    "\n- Pip cli:",
    pip.stdout.decode('utf-8'),
    pip.stderr.decode('utf-8'),
    "\n- Git cli:",
    git.stdout.decode('utf-8'),
    git.stderr.decode('utf-8'),
    "\n-Jupyter cli:",
    jupyter.stdout.decode('utf-8'),
    jupyter.stderr.decode('utf-8'),
    '\n- Visual Studio Code:',
    vscode.stdout,
    vscode.stderr,
    "\nPython modules",
    "--------------\n",
    '- matplotlib: ' + matplotlib,
    '- Backend: ' + matplotlib_backend.stdout,
    matplotlib_backend.stderr,
    '- mumpy: ' + numpy,
    '- pandas: ' + pandas,
    '- pip: ' + pip_py,
    '- pytest: ' + pytest,
    '- hypothesis: ' + hypothesis
    )